# Fetching Data from Oanda API Endpoints

Oanda API endpoints can be found at http://developer.oanda.com/rest-live-v20/account-ep/

In [35]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
account_data = pd.read_csv("~/Documents/temp/oanda/testnet_key.txt", sep=" ", header=None)
API_KEY = account_data[0][2]
ACCOUNT_ID = account_data[0][1]
OANDA_URL = account_data[0][0]

In [3]:
session = requests.Session()

## Fetching Data from the Candles Endpoint

In [4]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [5]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [6]:
def set_candles_url(url, pair):
 return f"{url}/instruments/{pair}/candles"

In [7]:
pair_name = "EUR_USD"
url_candles = set_candles_url(OANDA_URL, pair_name)

In [8]:
response = session.get(url_candles, params=params, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 1562,
   'time': '2022-07-19T22:00:00.000000000Z',
   'bid': {'o': '1.02260', 'h': '1.02408', 'l': '1.02235', 'c': '1.02347'},
   'mid': {'o': '1.02283', 'h': '1.02416', 'l': '1.02273', 'c': '1.02356'},
   'ask': {'o': '1.02306', 'h': '1.02425', 'l': '1.02285', 'c': '1.02366'}},
  {'complete': True,
   'volume': 2066,
   'time': '2022-07-19T23:00:00.000000000Z',
   'bid': {'o': '1.02346', 'h': '1.02362', 'l': '1.02288', 'c': '1.02297'},
   'mid': {'o': '1.02356', 'h': '1.02370', 'l': '1.02298', 'c': '1.02306'},
   'ask': {'o': '1.02366', 'h': '1.02382', 'l': '1.02307', 'c': '1.02314'}},
  {'complete': True,
   'volume': 2933,
   'time': '2022-07-20T00:00:00.000000000Z',
   'bid': {'o': '1.02302', 'h': '1.02479', 'l': '1.02300', 'c': '1.02454'},
   'mid': {'o': '1.02310', 'h': '1.02488', 'l': '1.02309', 'c': '1.02462'},
   'ask': {'o': '1.02317', 'h': '1.02499', 'l': '1.02317', 'c': '1.02469'}},

## Fetching Data from the Instruments Endpoint

In [11]:
url_inst = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [12]:
response = session.get(url_inst, params=params, data=None, headers=None)

In [13]:
response.status_code

200

In [14]:
data = response.json() # dict_keys(['instruments', 'lastTransactionID'])

In [15]:
instruments_list = data["instruments"]

In [16]:
len(instruments_list)

129

In [17]:
instruments_list

[{'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'minimumTradeSize': '0.1',
  'maximumTrailingStopDistance': '10000.0',
  'minimumTrailingStopDistance': '5.0',
  'maximumPositionSize': '0',
  'maximumOrderUnits': '500',
  'marginRate': '0.1',
  'guaranteedStopLossOrderMode': 'DISABLED',
  'tags': [{'type': 'ASSET_CLASS', 'name': 'INDEX'}],
  'financing': {'longRate': '-0.0259',
   'shortRate': '-0.0241',
   'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
    {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
    {'dayOfWeek': 'FRIDAY', 'daysCharged': 3},
    {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
    {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}},
 {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'trade

In [18]:
# taking a look at the information that's available for every instrument
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [19]:
# list of keys we're intersted in getting 
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 
         'tradeUnitsPrecision', 'marginRate']

In [20]:
instruments_dict = {}
for i in instruments_list:
    # print(i['name'])
    key = i['name']
    instruments_dict[key] = {
        k: i[k] for k in key_i
    }

In [21]:
instruments_dict

{'CHINAH_HKD': {'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'AU200_AUD': {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.05'},
 'USD_PLN': {'name': 'USD_PLN',
  'type': 'CURRENCY',
  'displayName': 'USD/PLN',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'GBP_AUD': {'name': 'GBP_AUD',
  'type': 'CURRENCY',
  'displayName': 'GBP/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'SG30_SGD': {'name': 'SG30_SGD',
  'type': 'CFD',
  'displayName': 'Singapore 30',
  'pipLocation': -1,
  'displayPrecision': 2,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'EUR_USD': {'name': 'EUR_USD',
  'type': 'CURRENCY',
  'displayName': 'EUR/USD',
  

In [22]:
# fetching particular instrument
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [23]:
with open("Data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

## Fetching a candle pair

In [24]:
def fetch_candles(api_url, pair_name, count="10", granularity="H1"):
    url = f"{api_url}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    status = response.status_code
    
    if status == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    
    return status, data

In [25]:
status, data = fetch_candles(OANDA_URL, "EUR_USD", count=20)

In [26]:
status

200

In [27]:
len(data)

20

In [28]:
data

[{'complete': True,
  'volume': 8116,
  'time': '2022-07-19T12:00:00.000000000Z',
  'bid': {'o': '1.02528', 'h': '1.02536', 'l': '1.02240', 'c': '1.02280'},
  'mid': {'o': '1.02535', 'h': '1.02543', 'l': '1.02248', 'c': '1.02288'},
  'ask': {'o': '1.02542', 'h': '1.02550', 'l': '1.02255', 'c': '1.02297'}},
 {'complete': True,
  'volume': 9162,
  'time': '2022-07-19T13:00:00.000000000Z',
  'bid': {'o': '1.02278', 'h': '1.02560', 'l': '1.02277', 'c': '1.02407'},
  'mid': {'o': '1.02286', 'h': '1.02568', 'l': '1.02286', 'c': '1.02415'},
  'ask': {'o': '1.02293', 'h': '1.02576', 'l': '1.02293', 'c': '1.02423'}},
 {'complete': True,
  'volume': 12796,
  'time': '2022-07-19T14:00:00.000000000Z',
  'bid': {'o': '1.02408', 'h': '1.02634', 'l': '1.02282', 'c': '1.02450'},
  'mid': {'o': '1.02415', 'h': '1.02648', 'l': '1.02290', 'c': '1.02456'},
  'ask': {'o': '1.02422', 'h': '1.02661', 'l': '1.02297', 'c': '1.02463'}},
 {'complete': True,
  'volume': 7258,
  'time': '2022-07-19T15:00:00.000000

In [36]:
# collecting certain info from the instruments
# and converting it to a data frame
final_data = []
for candle in data:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = parser.parse(candle['time'])
    new_dict['volume'] = candle['volume']    
    final_data.append(new_dict)
df = pd.DataFrame.from_dict(final_data)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    19 non-null     datetime64[ns, tzutc()]
 1   volume  19 non-null     int64                  
dtypes: datetime64[ns, tzutc()](1), int64(1)
memory usage: 432.0 bytes


In [34]:
df

,time,volume
0,2022-07-19T12:00:00.000000000Z,8116
1,2022-07-19T13:00:00.000000000Z,9162
2,2022-07-19T14:00:00.000000000Z,12796
3,2022-07-19T15:00:00.000000000Z,7258
4,2022-07-19T16:00:00.000000000Z,4424
5,2022-07-19T17:00:00.000000000Z,3402
6,2022-07-19T18:00:00.000000000Z,3665
7,2022-07-19T19:00:00.000000000Z,5434
8,2022-07-19T20:00:00.000000000Z,1743
9,2022-07-19T21:00:00.000000000Z,1009
